## Inject noise into age and los_er features

In [2]:
import sys
sys.path.append("..")
import mlflow
import pandas as pd
import config.config as config
import tasks.datapipeline.process_data as process_data 
import numpy as np 

mlflow.set_tracking_uri('../mlruns')

In [3]:
#add noise to lor_er and age (for demo purposes)
conf = None
conf = config.read_config('../config/gemini.cfg')
data, _ = process_data.pipeline(conf)

print(data['age'].head)

mu, sigma = 5, 3 
# creating a noise with the same dimension as the dataset  
noise = np.random.normal(mu, sigma, len(data['age'])).astype(int) 
print(noise)

data['age'] = data['age'].add(noise)
print(data['age'].head)

#repeat for los_er
mu, sigma = 5, 5 
# creating a noise with the same dimension as the dataset  
noise = np.random.normal(mu, sigma, len(data['los_er'])).astype(int) 
print(noise)

data['los_er'] = data['los_er'].add(noise)
data.to_csv('/mnt/nfs/project/delirium/data/all_before_2018_with_noise.csv')

conf.input = '/mnt/nfs/project/delirium/data/all_before_2018_with_noise.csv'
data, _ = process_data.pipeline(conf)
print(data['age'].head)

<bound method NDFrame.head of 0         87
1         52
2         59
3         62
4         52
          ..
281560    54
281561    85
281562    53
281563    80
281564    86
Name: age, Length: 281565, dtype: int64>
[ 4  6  0 ...  8 -1  4]
<bound method NDFrame.head of 0         91
1         58
2         59
3         66
4         58
          ..
281560    58
281561    89
281562    61
281563    79
281564    90
Name: age, Length: 281565, dtype: int64>
[11  8  0 ...  7  0  0]
<bound method NDFrame.head of 0         87
1         52
2         59
3         62
4         52
          ..
281560    54
281561    85
281562    53
281563    80
281564    86
Name: age, Length: 281565, dtype: int64>


## Explore extracted labs and outcome data

In [22]:
#from Bret's code
import pandas as pd
import os
HOSPITAL_ID = {'THPM':0, 'SBK':1, 'UHNTG':2, 'SMH':3, 'UHNTW':4, 'THPC':5, 'PMH':6, 'MSH':7}
HOSPITAL_ID_R = {v:k for k,v in HOSPITAL_ID.items()}
print(HOSPITAL_ID_R)

def load_data(data_dir, sites=None):
    filename = os.path.join(data_dir, 'all_hourly_data.h5')

    assert os.path.exists(filename), f"The file {filename} does not exist"


    dfs={}
    #dfs['statics'] = pd.read_hdf(filename, key='patients')
    

    if sites is None:
        return
    assert isinstance(sites, list)
    # then subset statics to sites:
    #dfs['statics']=dfs['statics'].loc[dfs['statics']['hospital_id'].isin(sites)]


    for site in sites:
        print(f'loading {site}')
        site_name = HOSPITAL_ID_R[site]
        dfs[f'numerics_{site}'] = pd.read_hdf(filename, key=f'vitals_labs_{site_name}')
        dfs[f'outcomes_{site}'] = pd.read_hdf(filename, key=f'interventions_{site_name}')
        # eliminate genc_ids at a site that are not in numerics.
        #numerics_genc_ids= set(dfs[f'numerics_{site}'].index.get_level_values('genc_id')) # get genc ids from particular site that have numerics data
        #[not(numerics_genc_id)&(hospital_id==site)]
        # this should only be done if you want to keep GIM patients
        #drop_index = dfs['statics'].loc[(~dfs['statics'].index.get_level_values('genc_id').isin(numerics_genc_ids))&(dfs['statics']['hospital_id']==int(site))].index.get_level_values('genc_id') # get the genc_id of the people not in the numerics genc_id but have data at the site
        #dfs.update({'statics':dfs['statics'].loc[~dfs['statics'].index.get_level_values('genc_id').isin(drop_index)]})
        

    

    return dfs

data = load_data('/mnt/nfs/project/delirium/data/', [3])

{0: 'THPM', 1: 'SBK', 2: 'UHNTG', 3: 'SMH', 4: 'UHNTW', 5: 'THPC', 6: 'PMH', 7: 'MSH'}
loading 3


In [23]:
print(data.keys())
data['numerics_3'].head()

dict_keys(['numerics_3', 'outcomes_3'])


(% free testosterone, %)  \
                                                                                                     mask   
patient_id                                         genc_id  hours_in hospital_id                            
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0                                0   
                                                            12       3.0                                0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0                                0   
                                                            6        3.0                                0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0                                0   

                                                                                                \
                                                                                   measurement   
patient_id                                         genc_id  hours_in hospital_id                 
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0                   NaN   
                                                            12       3.0                   NaN   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0          3.754579e+06   
                                                            6        3.0          3.754579e+06   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0          3.754579e+06   

                                                                                       \
                                                                                 time   
patient_id                                         genc_id  hours_in hospital_id        
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0          0.0   
                                                            12       3.0          0.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0          0.0   
                                                            6        3.0          0.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0          0.0   

                                                                                 (1,25 dihydroxy vitamin d, pmol/l)  \
                                                                                                               mask   
patient_id                                         genc_id  hours_in hospital_id                                      
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0                                          0   
                                                            12       3.0                                          0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0                                          0   
                                                            6        3.0                                          0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0                                          0   

                                                                                                \
                                                                                   measurement   
patient_id                                         genc_id  hours_in hospital_id                 
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0                   NaN   
                                                            12       3.0                   NaN   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0          4.192400e+06   
                                                            6        3.0          4.192400e+06   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.

In [24]:
data['outcomes_3'].head()

mort_24  \
patient_id                                         genc_id  hours_in hospital_id            
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0              0.0   
                                                            12       3.0              0.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0              0.0   
                                                            6        3.0              0.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0              0.0   

                                                                                  mort_48  \
patient_id                                         genc_id  hours_in hospital_id            
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0              0.0   
                                                            12       3.0              0.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0              0.0   
                                                            6        3.0              0.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0              0.0   

                                                                                  disch_24  \
patient_id                                         genc_id  hours_in hospital_id             
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0               0.0   
                                                            12       3.0               1.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0               0.0   
                                                            6        3.0               1.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0               0.0   

                                                                                  disch_48  \
patient_id                                         genc_id  hours_in hospital_id             
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0               1.0   
                                                            12       3.0               1.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0               0.0   
                                                            6        3.0               1.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0               0.0   

                                                                                  resus_24  \
patient_id                                         genc_id  hours_in hospital_id             
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0               0.0   
                                                            12       3.0               0.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0               0.0   
                                                            6        3.0               0.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0               0.0   

                                                                                  resus_48  \
patient_id                                         genc_id  hours_in hospital_id             
000af32bb1b1d265dfc933ec20cb52bbd10c5c4c60b490a... 11753022 6        3.0               0.0   
                                                            12       3.0               0.0   
000d6a78aa9311f7b13fd0cef2f554fbdfd993041dec83c... 11974941 0        3.0               0.0   
                                                            6        3.0               0.0   
000ff822746a56a05dc089831d21da20d8049b9602d6582... 11768685 6        3.0               0.0   

                                                                                  acute_24  \
patient_id                                      